In [2]:
import pandas as pd
import os

os.getcwd()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/narensham/Documents/Projects/Cursor/buildcpg-labs/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/narensham/Documents/Projects/Cursor/buildcpg-labs/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/narensham/Documents/Projects/Cursor/buildcpg-labs/.ve

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/narensham/Documents/Projects/Cursor/buildcpg-labs/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/narensham/Documents/Projects/Cursor/buildcpg-labs/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/narensham/Documents/Projects/Cursor/buildcpg-labs/.ve

AttributeError: _ARRAY_API not found

'/Users/narensham/Documents/Projects/Cursor/buildcpg-labs/notebooks/exploration'

In [3]:
from pathlib import Path
import pandas as pd

# Current notebook folder
NOTEBOOK_DIR = Path().resolve()  # e.g., .../notebooks/exploration

# Project root is 2 levels up
PROJECT_ROOT = NOTEBOOK_DIR.parent.parent

CSV_PATH = PROJECT_ROOT / "data/raw/sales_2025_09.csv"
print("Resolved CSV path:", CSV_PATH)

# Load CSV
df_raw = pd.read_csv(CSV_PATH)
df_raw.head()



Resolved CSV path: /Users/narensham/Documents/Projects/Cursor/buildcpg-labs/data/raw/sales_2025_09.csv


,order_id,sku_id,order_date,region,sales,promo_flag,discount,stock_on_hand
0,1,SKU_A,2025-09-01,North,100,0,0,50
1,2,SKU_A,2025-09-02,North,120,1,10,40
2,3,SKU_B,2025-09-01,South,80,0,0,70
3,4,SKU_B,2025-09-03,South,90,1,5,60
4,5,SKU_C,2025-09-02,West,60,0,0,20


In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   order_id       5 non-null      int64 
 1   sku_id         5 non-null      object
 2   order_date     5 non-null      object
 3   region         5 non-null      object
 4   sales          5 non-null      int64 
 5   promo_flag     5 non-null      int64 
 6   discount       5 non-null      int64 
 7   stock_on_hand  5 non-null      int64 
dtypes: int64(5), object(3)
memory usage: 452.0+ bytes


In [5]:
df_raw.describe()

,order_id,sales,promo_flag,discount,stock_on_hand
count,5.000000,5.00000,5.000000,5.000000,5.000000
mean,3.000000,90.00000,0.400000,3.000000,48.000000
std,1.581139,22.36068,0.547723,4.472136,19.235384
min,1.000000,60.00000,0.000000,0.000000,20.000000
25%,2.000000,80.00000,0.000000,0.000000,40.000000
50%,3.000000,90.00000,0.000000,0.000000,50.000000
75%,4.000000,100.00000,1.000000,5.000000,60.000000
max,5.000000,120.00000,1.000000,10.000000,70.000000


In [6]:
df_raw.isnull().sum()

order_id         0
sku_id           0
order_date       0
region           0
sales            0
promo_flag       0
discount         0
stock_on_hand    0
dtype: int64

In [8]:
import duckdb

db_path = PROJECT_ROOT / "data/buildcpg.duckdb"
con = duckdb.connect(db_path)

In [21]:
# See existing tables
tables = con.execute("SHOW TABLES").fetchall()
print("Tables in DuckDB:", tables)


Tables in DuckDB: [('bronze_sales',), ('df_raw',), ('sales_forecast_ready',), ('stg_sales',)]


In [28]:
import duckdb
import pandas as pd
from pathlib import Path

# Paths
NOTEBOOK_DIR = Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent.parent
CSV_PATH = PROJECT_ROOT / "data/raw/sales_2025_09.csv"

# Load CSV
df_raw = pd.read_csv(CSV_PATH)

# Connect to DuckDB
con = duckdb.connect(PROJECT_ROOT / "data/buildcpg.duckdb")

# Drop any existing object with the same name

con.execute("DROP VIEW IF EXISTS bronze_sales CASCADE")

# Register DataFrame
con.register("tmp_df", df_raw)

# Create persistent bronze table
con.execute("CREATE TABLE bronze_sales AS SELECT * FROM tmp_df")

# Verify
print("Rows in bronze_sales:", con.execute("SELECT COUNT(*) FROM bronze_sales").fetchone()[0])


Rows in bronze_sales: 5


In [31]:
# Preview first 5 rows in bronze
df_bronze = con.execute("SELECT * FROM bronze_sales LIMIT 5").df()
df_bronze


,order_id,sku_id,order_date,region,sales,promo_flag,discount,stock_on_hand
0,1,SKU_A,2025-09-01,North,100,0,0,50
1,2,SKU_A,2025-09-02,North,120,1,10,40
2,3,SKU_B,2025-09-01,South,80,0,0,70
3,4,SKU_B,2025-09-03,South,90,1,5,60
4,5,SKU_C,2025-09-02,West,60,0,0,20


In [32]:

# Check nulls in critical columns
con.execute("SELECT COUNT(*) FROM bronze_sales WHERE order_id IS NULL").fetchall()




[(0,)]

In [33]:
# Check date column (if exists) for sanity
con.execute("SELECT MIN(order_date), MAX(order_date) FROM bronze_sales").fetchall()

[('2025-09-01', '2025-09-03')]